# Solución

Ahora bien, se conoce que el logaritmo natural de la presión de saturación de un componente puro varía con el inverso de la temperatura absoluta.

\begin{equation}
\ln P^\text{sat} = A - \frac{B}{T+C}
\end{equation}

Si se linealiza esta expresión:

\begin{equation}
\ln P = A + \left(AC - B\right)\frac{1}{T} -  C \:\frac{\ln P}{T}
\end{equation}

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import leastsq

In [ ]:
Data_experimental = pd.read_excel('data_coef_antoine.xlsx', header=None, names=['T (K)', 'Psat (bar)'])

In [ ]:
Data_experimental

In [ ]:
Data = Data_experimental.values
T = Data[:,0] 
P =  Data[:,1]
x = 1/T
y = np.log(P)

In [ ]:
plt.clf()
fig, ax = plt.subplots()
_ = plt.plot(x, y, 'o')

In [ ]:
beta = np.polyfit(x, y, 1)

In [ ]:
fx = np.array([x.min(), x.max()])
fy = beta[1] + beta[0] * fx

plt.clf()
_ = plt.plot(x, y, 'o')
_ = plt.plot(fx, fy, '-', alpha=0.7)

In [ ]:
B = np.log(Data[:,1]).reshape((-1,1))
print(B)

In [ ]:
solucion = np.linalg.solve(A, B)

In [ ]:
solucion = np.linalg.solve(a,b)
a = solucion[0]
c = -solucion[2]
b = a*c - solucion[1]

print("Las constantes de la ecuacion de Antoine son: A = {:.2f}, B = {:.2f} y C = {:.2f}".format(A, B, C))

Ahora para determinar el calor de vaporizacion se relaciona la ecuacion de Antoine con la ecuación de Clausius-Clapeyron

\begin{align}
\left(\frac{\partial P^\text{sat}}{\partial T}\right)_{\Delta G =0 } & = \frac{\Delta H^\text{vap}}{R\,T^2} \notag \\
\frac{B}{\left( T + C \right)^2} & = \frac{\Delta H^\text{vap}}{R\,T^2} 
\end{align}

In [ ]:
R = 8.31416
Pfeed = 1.88e6
calidad = .9
PM = 58

Tfeed, = fsolve(lambda x : np.log(Pfeed) - (A - B/(x + C)), 350)

print("La temperatura de alimentacion es de {:.2f}K".format(Tfeed))

In [ ]:
DHvap, = fsolve(lambda x: B/(Tfeed + C)**2 - x/R/Tfeed**2, 20000)
print("La carga calorica en el vaporizador es {:.2f} J/g".format(DHvap * calidad / PM))